In [1]:
import pandas as pd

# LEITURA DE DADOS

# Leitura da planilha Dados_cliente1 (FATURA)
df_fatura = pd.read_excel('../data/raw/Dados_cliente1.xlsx')

# Leitura da planilha Dados_cliente1 (INFORMACOES)
df_informacoes = pd.read_excel('../data/raw/Dados_cliente1.xlsx', sheet_name=1)

# Leitura da planilha HSP-RS
df_hsp = pd.read_excel('../data/raw/HSP-RS.xlsx')

display(df_fatura.describe())

,MÊS/ANO,CONSUMO (KWH)
count,12,12.000000
mean,2025-06-16 12:00:00,370.083333
min,2025-01-01 00:00:00,228.000000
25%,2025-03-24 06:00:00,318.500000
50%,2025-06-16 00:00:00,373.000000
75%,2025-09-08 12:00:00,415.250000
max,2025-12-01 00:00:00,574.000000
std,NaN,91.479216


In [2]:
# CALCULAR MÉDIA APARADA COM BASE EM UMA NOVA COLUNA

# Cria nova coluna para remover os valores máximo e mínimo
df_fatura['CONSUMO (KWH) APARADO'] = df_fatura['CONSUMO (KWH)']

# Valor max = 0
max_index = df_fatura['CONSUMO (KWH) APARADO'].idxmin()
df_fatura.loc[max_index, 'CONSUMO (KWH) APARADO'] = 0

#Valor min = 0
min_index = df_fatura['CONSUMO (KWH) APARADO'].idxmax()
df_fatura.loc[min_index, 'CONSUMO (KWH) APARADO'] = 0

display(df_fatura)

# Calcula média aparada desconsiderando linhas com consumo = 0 
print(df_fatura[df_fatura['CONSUMO (KWH) APARADO'] != 0] ['CONSUMO (KWH) APARADO'].mean())

,MÊS/ANO,CONSUMO (KWH),CONSUMO (KWH) APARADO
0,2025-01-01,574,0
1,2025-02-01,410,410
2,2025-03-01,372,372
3,2025-04-01,431,431
4,2025-05-01,374,374
5,2025-06-01,352,352
6,2025-07-01,319,319
7,2025-08-01,228,0
8,2025-09-01,247,247
9,2025-10-01,385,385


363.9
